In [1]:
# https://pytorch.org/get-started/locally/
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
!pip install -U sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/e4/c3/a21a75dd2de8114a6876f16a36b033e3e62f8ade68085a711b24f4b57c17/torchvision-0.18.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for torchaudio from https://files.pythonhosted.org/packages/7a/bf/5af72c1c4522bcf67df140427120b0e7898b2abc5afa5da917b722983a5b/torchaudio-2.3.1-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ------ --------------------------------- 0.2/1.2 MB 4.1 MB/s eta 0:00:01
   ------------------------ --------------- 0.7/1.2 MB 7.6 MB/s eta 0:00:01
   ----------------------------- ---------- 0.9/1.2 MB 7.0 MB/s eta 0:00:01
   ---------------------------------- ----- 1.0/1.2 MB 5.4 MB/s eta 0:00:01
   -------------------------------

In [2]:
import torch
torch.__version__

'2.3.1+cpu'

In [3]:
from sentence_transformers import SentenceTransformer, util
import os
import time
import pandas as pd

C:\Users\HP\AppData\Roaming\Python\Python311\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [5]:
df = pd.read_csv('quora_duplicate_questions.tsv', sep='\t', nrows=20000)


In [6]:
df.shape

(20000, 6)

In [7]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [8]:
corpus_senteces = list(set(df['question1'].tolist() + df['question2'].tolist()))
len(corpus_senteces)

37766

In [9]:
model = SentenceTransformer('quora-distilbert-multilingual')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

C:\Users\HP\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HP\.cache\huggingface\hub\models--sentence-transformers--quora-distilbert-multilingual. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.77k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/447 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
corpus_embeddings = model.encode(corpus_senteces, show_progress_bar=True, convert_to_tensor=True)

Batches:   0%|          | 0/1181 [00:00<?, ?it/s]

In [11]:
model._target_device

`SentenceTransformer._target_device` has been deprecated, please use `SentenceTransformer.device` instead.


device(type='cpu')

In [12]:
corpus_embeddings = corpus_embeddings.to(model._target_device)

`SentenceTransformer._target_device` has been deprecated, please use `SentenceTransformer.device` instead.


In [ ]:
while True:
  query = input("Please enter a question: ")

  if query == 'n':
    break

  question_embedding = model.encode(query, convert_to_tensor=True)
  hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=5)
  hits = hits[0]
  print(hits)

  for hit in hits:
    print(hit['score'], ": ", corpus_senteces[hit['corpus_id']])

  print('\n\n\n\n')
  

Please enter a question: what is python
[{'corpus_id': 20032, 'score': 0.9261704087257385}, {'corpus_id': 29723, 'score': 0.9184615612030029}, {'corpus_id': 34277, 'score': 0.9140186905860901}, {'corpus_id': 25443, 'score': 0.9131043553352356}, {'corpus_id': 8669, 'score': 0.9106530547142029}]
0.9261704087257385 :  What are some cool python scripts?
0.9184615612030029 :  What is the difference between () and [] in Python?
0.9140186905860901 :  What are the differences between Python 1, 2, 3?
0.9131043553352356 :  The world's largest dinosaur discovered is a titanosaur. How big was his poop?
0.9106530547142029 :  What is the best beginner friendly book on python?





Please enter a question: how to learn python
[{'corpus_id': 8436, 'score': 0.9927080869674683}, {'corpus_id': 2602, 'score': 0.9891510605812073}, {'corpus_id': 57, 'score': 0.982798159122467}, {'corpus_id': 2533, 'score': 0.9637923240661621}, {'corpus_id': 35871, 'score': 0.9635709524154663}]
0.9927080869674683 :  What are